<a href="https://colab.research.google.com/github/kumar-abhishek/handson-ml2/blob/master/BachChorales_HandsOnChapter_15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# univariate lstm example
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
  X, y = list(), list()
  for i in range(len(sequence)):
    # find the end of this pattern
    end_ix = i + n_steps
    # check if we are beyond the sequence
    if end_ix > len(sequence)-1:
     break
    # gather input and output parts of the pattern
    seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
    X.append(seq_x)
    y.append(seq_y)
  return array(X), array(y)
# define input sequence
raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]
# choose a number of time steps
n_steps = 3
# split into samples
X, y = split_sequence(raw_seq, n_steps)
print(X)
print(y)

[[10 20 30]
 [20 30 40]
 [30 40 50]
 [40 50 60]
 [50 60 70]
 [60 70 80]]
[40 50 60 70 80 90]


In [0]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
# define model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features))) model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=200, verbose=0)
# demonstrate prediction
x_input = array([70, 80, 90])
x_input = x_input.reshape((1, n_steps, n_features))
# demonstrate prediction
x_input = array([70, 80, 90])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
                                                                                   
print(yhat)

In [0]:
# multivariate output stacked lstm example
from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
  X, y = list(), list()
  for i in range(len(sequences)):
    # find the end of this pattern
    end_ix = i + n_steps
    # check if we are beyond the dataset
    if end_ix > len(sequences)-1:
     break
    # gather input and output parts of the pattern
    seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix, :]
    X.append(seq_x)
    y.append(seq_y)
    return array(X), array(y)
# define input sequence
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])
# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))
# horizontally stack columns
dataset = hstack((in_seq1, in_seq2, out_seq))
print(dataset)
# choose a number of time steps
n_steps = 3
# convert into input/output
X, y = split_sequences(dataset, n_steps)
print(X)
print(y)
print(type(X))
# the dataset knows the number of features, e.g. 2
n_features = X.shape[2]
print(n_features)

[[ 10  15  25]
 [ 20  25  45]
 [ 30  35  65]
 [ 40  45  85]
 [ 50  55 105]
 [ 60  65 125]
 [ 70  75 145]
 [ 80  85 165]
 [ 90  95 185]]
[[[10 15 25]
  [20 25 45]
  [30 35 65]]]
[[40 45 85]]
<class 'numpy.ndarray'>
3


In [0]:
# define model
model = Sequential()
model.add(LSTM(100, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
model.add(LSTM(100, activation='relu')) 
model.add(Dense(n_features)) 
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=1000, verbose=0)


In [0]:
# demonstrate prediction
x_input = array([[70,75,145], [80,85,165], [90,95,185]])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

[[197.93314 223.3317  407.88092]]


**Q10[part-1]. Download the Bach chorales dataset and unzip it. It is composed of 382 chorales composed by Johann Sebastian Bach. Each chorale is 100 to 640 time steps long, and each time step contains 4 integers, where each integer corresponds to a note’s index on a piano (except for the value 0, which means that no note is played). Train a model—recurrent, convolutional, or both—that can predict the next time step (four notes), given a sequence of time steps from a chorale. **bold text**

In [4]:
from google.colab import files
uploaded = files.upload()

Saving chorale_000.csv to chorale_000.csv


In [5]:
import pandas as pd
X_train_000 = pd.read_csv('chorale_000.csv')
print(X_train_000.head(10))

   note0  note1  note2  note3
0     74     70     65     58
1     74     70     65     58
2     74     70     65     58
3     74     70     65     58
4     75     70     58     55
5     75     70     58     55
6     75     70     60     55
7     75     70     60     55
8     77     69     62     50
9     77     69     62     50


In [6]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
import numpy as np
X_train_000 = X_train_000.to_numpy()
dataX=[]
dataY=[]
for row in range(len(X_train_000)-1):
    dataX.append(X_train_000[row])
    dataY.append(X_train_000[row+1])
print(dataX[0:5])
print(dataY[0:5])

[array([74, 70, 65, 58]), array([74, 70, 65, 58]), array([74, 70, 65, 58]), array([74, 70, 65, 58]), array([75, 70, 58, 55])]
[array([74, 70, 65, 58]), array([74, 70, 65, 58]), array([74, 70, 65, 58]), array([75, 70, 58, 55]), array([75, 70, 58, 55])]


In [9]:
"""X = np.reshape(dataX, (len(dataX), 1, 4))
X_train, X_test, y_train,  y_test = train_test_split(X, dataY, test_size=0.33, random_state=42)
print(X_train[0:5])
print(y_train[0:5])
"""
# choose a number of time steps
n_steps = 3
# convert into input/output
i=0
X=[]
y=[]
while i+2<len(dataX):
  #multi-input
  X.append(np.array([dataX[i], dataX[i+1], dataX[i+2]]).tolist())
  y.append(dataX[i+1])
  i += 1
  
X=np.asarray(X)
y=np.asarray(y)
print(X[0:2])
print(y[0:2])


[[[74 70 65 58]
  [74 70 65 58]
  [74 70 65 58]]

 [[74 70 65 58]
  [74 70 65 58]
  [74 70 65 58]]]
[[74 70 65 58]
 [74 70 65 58]]


In [11]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
# the dataset knows the number of features, e.g. 2
n_features = X.shape[2]
print(n_features)

# choose a number of time steps
n_steps = 3

# define model
model = Sequential()
model.add(LSTM(100, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
model.add(LSTM(100, activation='relu')) 
model.add(Dense(n_features)) 
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=100, verbose=0)

4





Using TensorFlow backend.



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where










In [12]:
# demonstrate prediction
#print(X_train_000[0:5])
x_input = X[5]
print(x_input)
x_input = x_input.reshape((1, n_steps, n_features))
print(x_input)
yhat = model.predict(x_input, verbose=0)
print(yhat)

[[75 70 58 55]
 [75 70 60 55]
 [75 70 60 55]]
[[[75 70 58 55]
  [75 70 60 55]
  [75 70 60 55]]]
[[76.28318  68.759834 59.762547 54.700706]]


**Q10[part-2] Then use this model to generate Bach-like music, one note at a time: you can do this by giving the model the start of a chorale and asking it to predict the next time step, then appending these time steps to the input sequence and asking the model for the next note, and so on. Also make sure to check out Google’s Coconet model, which was used for a nice Google doodle about Bach.**

In [14]:
# the dataset knows the number of features, e.g. 2
n_features = X.shape[2]
print(n_features)

# choose a number of time steps: random/variale increasing now
n_steps = None

# define model
model_var_input = Sequential()
model_var_input.add(LSTM(100, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
model_var_input.add(LSTM(100, activation='relu')) 
model_var_input.add(Dense(n_features)) 
model_var_input.compile(optimizer='adam', loss='mse')
# fit model
model_var_input.fit(X, y, epochs=100, verbose=0)

4


In [18]:
# choose a number of time steps
n_steps = None
# convert into input/output
i=0
n=1
X=[]
y=[]
while i+1<len(dataX):
  #multi-input
  X.append(np.array([dataX[i]]).tolist())
  y=dataX[i+1]
  i += 1
  X1=np.asarray(X)
  y1=np.asarray(y)
  # demonstrate prediction
  x_input = X1
  print('Input: ', x_input)
  print('---------------')
  print('Expected Output: ', y1)
  print('---------------')
  x_input = x_input.reshape((1, n, n_features))
  n+=1
  yhat = model_var_input.predict(x_input, verbose=1)
  print('Predicated Output: ', yhat)
  print("\n\n")
  if i>10:
    break

Input:  [[[74 70 65 58]]]
---------------
Expected Output:  [74 70 65 58]
---------------
1/1 [==============================] - 0s 2ms/step
Predicated Output:  [[10.548267  11.647018   6.950516   7.2514567]]



Input:  [[[74 70 65 58]]

 [[74 70 65 58]]]
---------------
Expected Output:  [74 70 65 58]
---------------
1/1 [==============================] - 0s 2ms/step
Predicated Output:  [[35.980934 34.137165 28.10568  25.292353]]



Input:  [[[74 70 65 58]]

 [[74 70 65 58]]

 [[74 70 65 58]]]
---------------
Expected Output:  [74 70 65 58]
---------------
1/1 [==============================] - 0s 1ms/step
Predicated Output:  [[76.00812  68.45716  64.09076  57.399487]]



Input:  [[[74 70 65 58]]

 [[74 70 65 58]]

 [[74 70 65 58]]

 [[74 70 65 58]]]
---------------
Expected Output:  [75 70 58 55]
---------------
1/1 [==============================] - 0s 1ms/step
Predicated Output:  [[134.52856 106.42965 117.29928  97.77108]]



Input:  [[[74 70 65 58]]

 [[74 70 65 58]]

 [[74 70 65 